In [1]:
print('hello')

hello


In [3]:
!which python

/opt/conda/bin/python


In [4]:
!python --version

Python 3.11.6


In [5]:
!java -version

openjdk version "17.0.8.1" 2023-08-24
OpenJDK Runtime Environment (build 17.0.8.1+1-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 17.0.8.1+1-Ubuntu-0ubuntu122.04, mixed mode, sharing)


In [6]:
!which java

/usr/bin/java


In [7]:
# spark home
import os
os.environ.get('SPARK_HOME')

'/usr/local/spark'

In [11]:
os.environ.get('JAVA_HOME')

In [8]:
os.environ.get('PYTHONPATH')

'/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip:/usr/local/spark/python:'

In [12]:
import pyspark

In [15]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pyspark example1').getOrCreate() #chaining
# sparkContext.SparkSession

In [16]:
spark

In [17]:
spark.stop()

In [ ]:
spark = SparkSession.builder.appName('pyspark example1').getOrCreate()

In [19]:
spark

In [20]:
data = [('Alice',1),('Bob',2),('Charlie',3)]
type(data)

list

In [21]:
# DataFrame 객체(부산객체)를 생성 <> 판다스의 데이터프레임이 아님
data1 = spark.createDataFrame(data, ['Name','Value'])
data1

DataFrame[Name: string, Value: bigint]

In [22]:
data1[1][1]

Column<'Value[1]'>

In [23]:
data1.show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+



# RDD 객체 생성

In [25]:
spark = SparkSession.builder.appName('pyspark example1').getOrCreate()

In [26]:
rdd = spark.sparkContext.parallelize([1,2,3,4,5])
rdd

ParallelCollectionRDD[7] at readRDDFromFile at PythonRDD.scala:289

In [27]:
data1

DataFrame[Name: string, Value: bigint]

In [29]:
rdd.take(5) #rdd 객체를 출력하는 함수 -n개를 지정

[1, 2, 3, 4, 5]

In [30]:
# map 연산: rdd값으로 연산
squared_rdd = rdd.map(lambda x : x*x)
squared_rdd

PythonRDD[12] at RDD at PythonRDD.scala:53

In [31]:
squared_rdd.take(5) # head랑 비슷

[1, 4, 9, 16, 25]

In [32]:
rdd.take(5)

[1, 2, 3, 4, 5]

In [33]:
squared_rdd.collect() #다 가져오는거 

[1, 4, 9, 16, 25]

In [35]:
data1.filter(data1.Name == 'Bob') # 결과물이 객체로 만들어짐

DataFrame[Name: string, Value: bigint]

In [37]:
data1.filter(data1.Name == 'Bob').show()

+----+-----+
|Name|Value|
+----+-----+
| Bob|    2|
+----+-----+



In [38]:
data1.filter(data1.Value > 2).show()

+-------+-----+
|   Name|Value|
+-------+-----+
|Charlie|    3|
+-------+-----+



In [39]:
data1.createOrReplaceTempView('people')

In [40]:
spark.sql('select * from people').show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+



In [43]:
spark.sql('select * from people where `Value` > 2').show()

+-------+-----+
|   Name|Value|
+-------+-----+
|Charlie|    3|
+-------+-----+



In [47]:
spark.sql('select * from people where Name = "Bob"').show()

+----+-----+
|Name|Value|
+----+-----+
| Bob|    2|
+----+-----+



# MLlib

In [48]:
from pyspark.ml.regression import LinearRegression

In [50]:
from pyspark.ml.feature import VectorAssembler

In [51]:
data_age = [('Alice',25),('Bob',30),('Charlie',33)]

In [53]:
data2 = spark.createDataFrame(data_age, ['Name','Age'])
data2

DataFrame[Name: string, Age: bigint]

In [55]:
assembler = VectorAssembler(inputCols = ['Age'], outputCol = 'features')
vector_df = assembler.transform(data2)
vector_df

DataFrame[Name: string, Age: bigint, features: vector]

In [56]:
lr = LinearRegression(featuresCol = 'features', labelCol ='Age' )
model = lr.fit(vector_df)

In [58]:
pred = model.transform(vector_df)
pred

DataFrame[Name: string, Age: bigint, features: vector, prediction: double]

In [59]:
pred.show()

+-------+---+--------+-----------------+
|   Name|Age|features|       prediction|
+-------+---+--------+-----------------+
|  Alice| 25|  [25.0]|24.99999999999993|
|    Bob| 30|  [30.0]|30.00000000000001|
|Charlie| 33|  [33.0]|33.00000000000006|
+-------+---+--------+-----------------+



In [61]:
spark.stop()

# streaming

In [60]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [62]:
spark = SparkSession.builder.appName('pyspark example1').getOrCreate()

In [ ]:
lines = spark.readStream.format("socket")\ #lines는 스트리밍에서 읽어온 데이터 객체
    .option('host', 'loclahost')\
    .option('port', 9999).load()
# streaming data를 로드

In [ ]:
aaaa bbbb cccc hello

In [ ]:
words = lines.select(explode(split(lines.value, '')).alias('word')